<a href="https://colab.research.google.com/github/lerakutt2/myprojects/blob/main/%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Классификация

В этом блоке происходит дообучение модели BERT для классификации новостей.

На вход подается датасет со сбалансированными классами новостей (balanced_news.csv). Новости были взяты с сайта журнала [Naked science](https://naked-science.ru).

На выходе сохраняется дообученная модель (news_classificator1.pth).

## Предобработка датасета

In [ ]:
!pip install -U bitsandbytes

In [ ]:
import torch

# если выполняется на ускорителе (gpu)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
import pandas as pd

# читаем датасет со сбалансированными классами новостей
all_data = pd.read_csv("/content/balanced_news.csv", index_col=0)
all_data

,title,importance,description,date,topic
0,На краю Солнечной системы нашли кандидата в ка...,5.2,Объект совпадающий по характеристикам с карли...,2025-05-23T09:25:14+03:00,Астрономия
1,Молодые звезды «оживили» галактики — спутники ...,4.6,Астрономы долго считали небольшие сфероидальны...,2025-05-22T15:11:06+03:00,Астрономия
2,Подледный океан на Европе предложили найти по ...,4.8,Ученые экспериментально выяснили как обнаружи...,2025-05-22T10:39:13+03:00,Астрономия
3,Квазар сыграл роль «копья» в «битве» двух гала...,5.5,Квазары будучи одними из самых энергетически ...,2025-05-21T18:04:28+03:00,Астрономия
4,Галактику с необычно гигантской перемычкой наш...,NaN,В молодой Вселенной галактики очень быстро нар...,2025-05-21T18:01:27+03:00,Астрономия
...,...,...,...,...,...
3833,"Палатка с кроватью, Wi-Fi и солнечной батареей...",1.8,Группа дизайнеров и инженеров из ряда именитых...,2021-01-19T07:45:00+03:00,Технологии
3834,"«Убить» Windows иконкой: обнаружен баг, повреж...",7.0,Специалисты по информационной безопасности наш...,2021-01-16T17:27:52+03:00,Технологии
3835,Нейросеть подтвердила псевдонаучную гипотезу о...,6.9,Американский исследователь создал нейросеть к...,2021-01-15T17:44:27+03:00,Технологии
3836,"Исследование показало, что люди не смогут упра...",4.8,Международная группа ученых продемонстрировала...,2021-01-12T13:24:02+03:00,Технологии


In [ ]:
all_data.date = pd.to_datetime(all_data.date)
all_data.topic.value_counts()

,count
topic,
Астрономия,340
Биология,340
Медицина,340
Психология,340
Физика,340
Палеонтология,340
Антропология,340
Технологии,340
История,340


In [ ]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3838 entries, 0 to 3837
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype                    
---  ------       --------------  -----                    
 0   title        3838 non-null   object                   
 1   importance   3684 non-null   float64                  
 2   description  3838 non-null   object                   
 3   date         3838 non-null   datetime64[ns, UTC+03:00]
 4   topic        3838 non-null   object                   
dtypes: datetime64[ns, UTC+03:00](1), float64(1), object(3)
memory usage: 308.9+ KB


Предобрабатываем датасет: кодируем лейблы и оставляем только нужные для обучения колонки: заголовок (title), описание (description) и закодированный лейбл (label).

In [ ]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
labels = label_encoder.fit_transform(list(all_data.topic))
all_data['label'] = pd.DataFrame(labels)
df = all_data[['title', 'description', 'label']]
df

,title,description,label
0,На краю Солнечной системы нашли кандидата в ка...,Объект совпадающий по характеристикам с карли...,1
1,Молодые звезды «оживили» галактики — спутники ...,Астрономы долго считали небольшие сфероидальны...,1
2,Подледный океан на Европе предложили найти по ...,Ученые экспериментально выяснили как обнаружи...,1
3,Квазар сыграл роль «копья» в «битве» двух гала...,Квазары будучи одними из самых энергетически ...,1
4,Галактику с необычно гигантской перемычкой наш...,В молодой Вселенной галактики очень быстро нар...,1
...,...,...,...
3833,"Палатка с кроватью, Wi-Fi и солнечной батареей...",Группа дизайнеров и инженеров из ряда именитых...,9
3834,"«Убить» Windows иконкой: обнаружен баг, повреж...",Специалисты по информационной безопасности наш...,9
3835,Нейросеть подтвердила псевдонаучную гипотезу о...,Американский исследователь создал нейросеть к...,9
3836,"Исследование показало, что люди не смогут упра...",Международная группа ученых продемонстрировала...,9


Перемешиваем полученный датасет и разбиваем на две части: датасет, который будет использован в обучении (для тренировки и валидации) и тестовый датасет.

In [ ]:
df = df.sample(frac=1)

train_size = int(df.shape[0] * 0.9)
df_train_val = df.iloc[:train_size].reset_index(drop=True)
df_test = df.iloc[train_size:].reset_index(drop=True)
df_train_val

,title,description,label
0,Шимпанзе снимают стресс потиранием гениталий,Ближайшие родственники человека — бонобо и обы...,2
1,"Зоологи впервые увидели, как лягушки спарились...",Группа исследователей провела 55 ночей в вечно...,2
2,Палеонтологи открыли древнего гигантского дель...,Международная группа ученых объявила об открыт...,7
3,Особо суровые зимы в Северном полушарии связал...,Исследователи из США и Израиля 10 лет назад об...,4
4,Удаление миндалин в детстве связали с повышенн...,Может ли хирургическое удаление миндалин в ран...,6
...,...,...,...
3449,Ученые уточнили совет не смотреть в экраны гад...,Для поддержания гигиены сна рекомендуется за д...,6
3450,Социолог назвала общую деталь детства у школьн...,В прошлых исследованиях о скулшутинге в Соедин...,8
3451,В землях Израилевых готовили рыбу еще 780 тыся...,Израильские ученые рассказали о самом раннем п...,0
3452,Нейрохирурги впервые оценили опасность турниро...,В последние годы в интернете и на некоторых ТВ...,6


## Предобработка текста и обучение

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
input_ids = []
attention_mask = []
token_type_ids = []

for i in range(len(df_train_val)):
  # токенезируем каждое предложение
  tokenized_sent = tokenizer(
      df_train_val.title[i],
      df_train_val.description[i],
      max_length = 150,
      padding='max_length',
      truncation = True,
      return_tensors = 'pt'
      )
  # индексы токенов
  input_ids.append(tokenized_sent['input_ids'])
  # показывает, где само предложение, а где pad-токены
  attention_mask.append(tokenized_sent['attention_mask'])
  # показывает, где первое предложение, а где второе
  token_type_ids.append(tokenized_sent['token_type_ids'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_mask, dim=0)
token_type_ids = torch.cat(token_type_ids, dim=0)
train_val_labels = torch.tensor(df_train_val.label)

In [ ]:
from torch.utils.data import TensorDataset, random_split

# создаем тензорный датасет
dataset = TensorDataset(input_ids, attention_masks, token_type_ids, train_val_labels)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# разбиваем датасет на тренировочный и валидационный
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
print('{:>5,} training samples'.format(train_size))
print('{:>5} validation samples'.format(val_size))

3,108 training samples
  346 validation samples


In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased",
    num_labels = len(label_encoder.classes_),
    output_attentions = False,
    output_hidden_states = False,
                                  )
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

# создаем dataLoader-ы для удобной итерации по батчам
train_dataloader = DataLoader(
    train_dataset,
    sampler = RandomSampler(train_dataset),
    batch_size=batch_size
    )

validation_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset),
    batch_size=batch_size
    )


In [ ]:
from torch.optim import AdamW

# задаем оптимизатор
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-7 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 5

total_steps = len(train_dataloader) * epochs

# после определенного количества шагов понижает learning rate
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

Обучение

In [ ]:
import random

random.seed(42)

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    total_train_loss = 0

    model.train() # переводим модель в режим train, чтобы обновлять веса

    for step, batch in enumerate(train_dataloader):

        #   0: input ids
        #   1: attention masks
        #   2: token type ids
        #   3: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_type_ids = batch[2].to(device)
        b_labels = batch[3].to(device)

        model.zero_grad() # зануляем градиенты с предыдущего шага

        # forward pass
        res = model(b_input_ids,
                    token_type_ids=b_type_ids,
                    attention_mask=b_input_mask,
                    labels=b_labels)

        loss = res['loss'] # потери
        logits = res['logits'] # вероятности классов для батча

        total_train_loss += loss.item()

        loss.backward() # backward pass и подсчет градиентов

        # урезаем норму до единичной, чтобы предотвратить взрывающиеся градиенты
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step() # обновляем веса

        scheduler.step() # обновляем learning rate

    print("")
    print("  Average training loss: {0:.2f}".format(total_train_loss / len(train_dataloader)))

    # ========================================
    #                Validation
    # ========================================

    print("")
    print("Running Validation...")

    model.eval() # чтобы нельзя было менять веса

    eval_loss, eval_accuracy = 0, 0

    for batch in validation_dataloader:

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_type_ids = batch[2].to(device)
        b_labels = batch[3].to(device)

        # без подсчета гридиентов
        with torch.no_grad():
            res = model(b_input_ids,
                        token_type_ids=b_type_ids,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        loss = res['loss']
        logits = res['logits']

        eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        eval_accuracy += flat_accuracy(logits, label_ids)

    print("  Accuracy: {0:.2f}".format(eval_accuracy / len(validation_dataloader)))
    print("  Validation loss: {0:.2f}".format(eval_loss / len(validation_dataloader)))

print("")
print("Training complete!")


======== Epoch 1 / 5 ========
Training...

  Average training loss: 1.67

Running Validation...
  Accuracy: 0.78
  Validation loss: 0.86

======== Epoch 2 / 5 ========
Training...

  Average training loss: 0.62

Running Validation...
  Accuracy: 0.82
  Validation loss: 0.63

======== Epoch 3 / 5 ========
Training...

  Average training loss: 0.36

Running Validation...
  Accuracy: 0.84
  Validation loss: 0.51

======== Epoch 4 / 5 ========
Training...

  Average training loss: 0.23

Running Validation...
  Accuracy: 0.85
  Validation loss: 0.51

======== Epoch 5 / 5 ========
Training...

  Average training loss: 0.17

Running Validation...
  Accuracy: 0.85
  Validation loss: 0.52

Training complete!


In [ ]:
# Сохраняем обученную модель
# torch.save(model.state_dict(), 'news_classificator1.pth')

# Тест и заполнение данных

In [ ]:
import torch

# если выполняется на ускорителе (gpu)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
from sklearn import preprocessing

# Создаем точно такой же label encoder, как для обучения
labels = ['Антропология', 'Астрономия', 'Биология', 'История', 'Климат',
    'Космонавтика', 'Медицина', 'Палеонтология', 'Психология',
    'Технологии', 'Физика', 'Химия']
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit_transform(labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [ ]:
from sklearn.metrics import f1_score
from transformers import BertForSequenceClassification
from sklearn import preprocessing
from transformers import AutoTokenizer
from torch.utils.data import DataLoader, SequentialSampler
import pandas as pd
from torch.utils.data import TensorDataset, random_split

class Model:
  '''
  Содержит обученную модель, токенизатор, правильно закодированные лейблы.
  Параметры:
  model_path: путь к модели в файловой системе
  model: сама модель, если она уже загружена
  Может предсказать темы новостей.
  '''
  def __init__(self, model_path=None, model=None):
    '''
    Можно передать на вход один из двух параметров:
    model_path - путь к сохраненной модели
    model - сама модель
    '''
    self._load_model(model_path, model)

  def _load_model(self, path, model):
    if (model is None):
      if (path is None):
        raise Exception("Передайте в качестве параметров или модель, или путь к ней")
      else:
        self.model = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased",
        num_labels = len(label_encoder.classes_),
        output_attentions = False,
        output_hidden_states = False)

        self.model.load_state_dict(torch.load(path, weights_only=True))
        self.model.to(device)
        print("Модель загружена.")
    else: self.model = model

    self.tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")


  def predict(self, test_data, no_topics=False):
    ''' Предсказывает тестовую выборку, а также любой датасет, в котором есть колонки "title" и "description"'''
    # если предсказание выполняется для неизвестных тем,
    # то есть не для тестовой выборки, запоняем колонку нулями
    if no_topics:
      test_data['label'] = pd.Series(np.zeros(len(test_data)))

    # предобработка текста
    input_ids = []
    attention_mask = []
    token_type_ids = []

    for i in range(len(test_data)):
      tokenized_sent = self.tokenizer(
          test_data.title[i],
          test_data.description[i],
          max_length = 150,
          padding='max_length',
          truncation = True,
          return_tensors = 'pt'
          )

      input_ids.append(tokenized_sent['input_ids'])
      attention_mask.append(tokenized_sent['attention_mask'])
      token_type_ids.append(tokenized_sent['token_type_ids'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_mask = torch.cat(attention_mask, dim=0)
    token_type_ids = torch.cat(token_type_ids, dim=0)
    test_labels = torch.tensor(test_data.label)

    batch_size = len(test_data)

    # создаем DataLoader.
    prediction_data = TensorDataset(input_ids, attention_mask, token_type_ids, test_labels)
    prediction_sampler = SequentialSampler(prediction_data)
    prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

    print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))
    self.model.eval()
    predictions, true_labels = [], []

    # Предсказываем тестовую выборку
    for batch in prediction_dataloader:
      # to GPU
      batch = tuple(t.to(device) for t in batch)

      b_input_ids, b_input_mask, b_type_ids, b_labels = batch
      with torch.no_grad():
          # Forward pass
          outputs = self.model(b_input_ids, token_type_ids=b_type_ids,
                          attention_mask=b_input_mask)

      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()

      predictions.append(logits)
      true_labels.append(label_ids)

    print('    DONE.')

    flat_predictions = np.concatenate(predictions, axis=0)
    flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

    flat_true_labels = np.concatenate(true_labels, axis=0)
    f1_macro = f1_score(flat_true_labels, flat_predictions, average='macro')

    if not no_topics:
      print('Macro f1 score: %.3f' % f1_macro)

    return flat_true_labels, flat_predictions


  def preprocess_data(self, data):
    '''  Обрабатывает "сырой" датасет с данными новостей сайта Naked science '''
    # Переводим дату в формат datetime
    data.date = pd.to_datetime(data.date)

    all_topics = data.topic.unique()
    uncommon_topics = [x for x in all_topics if x not in label_encoder.classes_]
    # датасет новостей с известными темами
    df_marked = data[data.topic.isin(label_encoder.classes_)].reset_index(drop=True)
    # датасет новостей с NaN темами или редкими темами, которые модель не обучалась предсказывать
    df_unmarked = data[data.topic.isin(uncommon_topics)].reset_index(drop=True)

    # Предсказываем темы для df_unmarked
    _, pred_topics = self.predict(df_unmarked, no_topics=True)
    # Переводим темы из чисел обратно в текст
    pred_topics = label_encoder.inverse_transform(pred_topics)
    df_unmarked['topic'] = pd.Series(pred_topics)

    # Собираем новый датасет из новостей с изначально известными темами и с предсказанными
    complete_data = pd.concat([df_marked, df_unmarked], ignore_index=True).drop('label', axis=1)
    # Удаляем новости, у которых не был указан индекс важности (их единицы)
    complete_data = complete_data.dropna(subset='importance', ignore_index=True)

    return complete_data

In [ ]:
myModel = Model(model=model) # если до этого выполнялся блок с обучением

In [ ]:
# Если хотим загрузить модель из файлов

# path = "/content/news_classificator1.pth"
# myModel = Model(model_path=path)

Предсказываем тестовую выборку

In [ ]:
true_l, pred_l = myModel.predict(df_test)

Predicting labels for 384 test sentences...
    DONE.
Macro f1 score: 0.886


Macro f1 для тестовой выборки 0.86-0.88



In [ ]:
import pandas as pd

def fill_news():
  ''' Заполняет датасет с новостями '''
  data = pd.read_csv('/content/news.csv', index_col=0)
  complete_data = myModel.preprocess_data(data)
  complete_data.to_csv("complete_news.csv")
  return complete_data

In [ ]:
# Можно проверить как работает предыдущая функция
complete_data = fill_news()

Predicting labels for 1,279 test sentences...
    DONE.


# Функции для парсинга

Функции, предназначенные для сбора датасетов новостей с сайта Naked science. Каждая новость содержит:
- заголовок
- индекс важности
- описание
- дата публикации
- тема

Статьи собираются в хронологическом порядке.

In [ ]:
!pip install fake_useragent

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 7.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import time
import numpy as np
import requests
from fake_useragent import UserAgent

In [ ]:
from bs4 import BeautifulSoup

link = 'https://naked-science.ru/article/'

In [ ]:
def get_key(d, value):
  ''' Получение ключа по значению в словаре '''
  for k, v in d.items():
      if v == value:
          return k

In [ ]:
def parse_titles_importance(soup):
  ''' Читает все заголовки новостей и их индексы важности со страницы '''
  titles_html = soup.find_all(attrs = {'class':'news-item-title'})
  importance = []
  for t in titles_html:
    if t.h3.a.span == None:
      importance.append(None)
    else:
      importance.append(float(t.h3.a.span.extract().text))

  titles = [t.h3.a.text.replace('\xa0', ' ').rstrip() for t in titles_html]

  return titles, importance

In [ ]:
import re

def parse_desc(soup):
  ''' Считывает все описания новостей со страницы'''
  descr = soup.find_all(attrs = {'class':'news-item-excerpt'})
  description = [re.sub(r"['\xa0', '\n']", " ", d.p.text).rstrip() for d in descr]
  return description

In [ ]:
def parse_dates(soup):
  ''' Читает дату публикации '''
  dates_html = soup.find_all(lambda tag: tag.get('class') == ['echo_date'] and tag.parent.parent.get('class') == ['meta-items-l', 'grid'])
  dates = [d.attrs['data-published'] for d in dates_html]
  return dates

In [ ]:
def parse_topics(soup):
  ''' Читает тему новости '''
  topics_html = soup.find_all(lambda tag: tag.get('class') == ['terms-items', 'grid'] and tag.parent.parent.get('class') == ['news-item-left', 'with-bookmark'])
  topics = []

  for t in topics_html:
    if t.div['class'] == ['commnets-count']:
      obj = t.div.find_next_sibling()
    else: obj = t.div

    if obj.noindex == None:
      topics.append(None)
    elif obj.noindex.a == None:
      topics.append(None)
    elif obj.noindex.a['class'] == ['animate-custom', 'with-bg']:
      topics.append(obj.noindex.a.text)
    else:
      topics.append(None)

  return topics

In [ ]:
def parse_page(page='', topic=''):
  '''
  Парсит страницу с номером page.
  Если нужно найти страницу с новостями по определенной теме,
  Передается параметр topic - имя раздела в ссылке
  '''
  response = requests.get(url=link+topic+page, headers={'User-Agent': UserAgent().chrome})
  html = response.content
  soup = BeautifulSoup(html, 'html.parser')
  title_imp = parse_titles_importance(soup)
  if topic == '':
    topics = parse_topics(soup)
  else:
    topics = [get_key(links, topic)] * len(title_imp[0])

  data = {
      'title': title_imp[0],
      'importance': title_imp[1],
      'description': parse_desc(soup),
      'date': parse_dates(soup),
      'topic': topics,
  }
  return data

In [ ]:
def parse(page_count=1, page_start=1, topic_name='', date_from='', verbose=False):
  """
  Парсинг страниц с новостными статьями, начиная с самых ранних.
      Параметры
      ----------
      page_count: int, optional
        количество страниц, которые нужно распарсить. (>=1)
      page_start: int, optional
        номер страницы, начиная с которой нужно читать. (>=1)
      topic_name: str, optional
        имя раздела, как он написан в ссылке. Например, если topic_name='biology',
        то будут читаться страницы https://naked-science.ru/article/biology, https://naked-science.ru/article/biology/page/1...
        Все имена разделов с темами перечислены в словаре links
      date_from: datetime, optional
        дата, до которой производится чтение страниц
      varbose: bool, optional
        Выводить ли текст о работе функции
  """
  if verbose: print(f'Parsing page {1} from {page_count}...')
  data = parse_page(topic=topic_name) # парсим первую страницу
  for i in range(page_start, page_count):
    if verbose: print(f'Parsing page {i + 1} from {page_count}...')
    new_data = parse_page(page='/page/'+str(i + 1), topic=topic_name)
    if pd.to_datetime(min(new_data['date'])) < date_from:
      return data

    for key in data.keys():
      data[key] += new_data[key]

  return data

In [ ]:
links = {
  'Химия': 'chemistry',
  'Астрономия': 'astronomy',
  'Биология': 'biology',
  'Медицина': 'medicine',
  'Психология':	'psy',
  'Физика':	'physics',
  'Палеонтология':	'paleontology',
  'Космонавтика':	'cosmonautics',
  'Антропология':	'anthropology',
  'Климат': 'climate',
  'История': 'history',
  'Технологии': 'hi-tech'
}

# Суммаризация

In [ ]:
import torch

# если выполняется на ускорителе (gpu)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
import pandas as pd
df = pd.read_csv("/content/complete_news.csv", index_col=0)
df.date = pd.to_datetime(df.date)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2949 entries, 0 to 2948
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype                    
---  ------       --------------  -----                    
 0   title        2949 non-null   object                   
 1   importance   2949 non-null   float64                  
 2   description  2949 non-null   object                   
 3   date         2949 non-null   datetime64[ns, UTC+03:00]
 4   topic        2949 non-null   object                   
dtypes: datetime64[ns, UTC+03:00](1), float64(1), object(3)
memory usage: 138.2+ KB


In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

class Conversation:
    ''' Класс для работы с моделью Saiga '''
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import datetime

class Report:
  ''' Позволяет сделать репортаж о самых важных новостях, произошедших за определенный период '''
  def __init__(self, date_from, date_to, data=None, model_path=None, model=None, max_page_count=20, make_short_descr=False):
    '''
    data: датасет новостей с темами и датами (complete_news.csv)
    model_path: путь к обученной модели
    model: обученная модель (передается одно из двух или ничего)
    date_from - date_to: промежуток времени, за который нужно сделать репортаж
    '''
    self.date_from = date_from
    self.date_to = date_to
    self.data = data
    self.max_page_count = max_page_count
    if model is not None or model_path is not None:
      self.myModel = Model(model=model, model_path=model_path)
    self.get_data()
    self.make_short_descr = make_short_descr
    if make_short_descr: self._init_conversation_model()


  def get_data(self):
    if (self.data is None):
      # если датасета нет, то читаем все новости, начиная с нужной даты
      # page_count=20 значит, что если если достигнут лимит в 20 страниц, парсинг прекращается для экономии времени
      parsed_page = parse(date_from=self.date_from, page_count=self.max_page_count)

      # обрабатываем сырой датасет: назначаем темы, удаляем пропуски
      self.df = self.myModel.preprocess_data(pd.DataFrame(parsed_page))

      # оставляем только новости, которые были в указанный временной промежуток
      self.df = self.df[(self.df.date >= self.date_from) & (self.df.date <= self.date_to)].reset_index(drop=True)

    elif self.date_to > self.data.date.max():
      # если есть свежие новости
      parsed_page = parse(date_from=max(self.data.date.max(), self.date_from), page_count=self.max_page_count, verbose=True)
      new_news = myModel.preprocess_data(pd.DataFrame(parsed_page))
      self.df = pd.concat([new_news, self.data], ignore_index=True)
      self.df = self.df[(self.df.date >= self.date_from) & (self.df.date <= self.date_to)].reset_index(drop=True)
      self.df = self.df.drop_duplicates()
    else:
      self.df = self.data[(self.data.date >= self.date_from) & (self.data.date <= self.date_to)].reset_index(drop=True)

  def _init_conversation_model(self):
    config = PeftConfig.from_pretrained(MODEL_NAME)
    self.conv_model = AutoModelForCausalLM.from_pretrained(
        config.base_model_name_or_path,
        load_in_8bit=True,
        torch_dtype=torch.float16,
        device_map="cpu"
    )
    self.conv_model = PeftModel.from_pretrained(
        self.conv_model,
        MODEL_NAME,
        torch_dtype=torch.float16
    )
    self.conv_model.eval()

    self.conv_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
    self.generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

  def generate(self, model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

  def short_descr(self, descr):
    inp = "Сделай очень краткий перезсказ новости: " + descr
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(self.conv_tokenizer)
    print('generating...')
    output = self.generate(self.conv_model, self.conv_tokenizer, prompt, self.generation_config)
    return output

  def make_report(self, n=5):
    ''' Выдает основные новости по областям за период.
    n - максимальное количество новостей по теме '''
    if self.df.empty: print(f"Новостей за период с {self.date_from} по {self.date_to} нет")
    elif not self.make_short_descr:
      topics = self.df.topic.unique()
      print(f"Cамые важные новости с {self.df.date.min()} по {self.df.date.max()}")
      for t in topics:
        print(f"По теме {t} произошло:")
        temp_df = self.df[self.df.topic == t].sort_values(by = 'importance', ascending=False).head(n).reset_index(drop=True)
        for i, row in temp_df.iterrows():
          print(f"{i+1}. {row.title}")
    else:
      try:
        topics = self.df.topic.unique()
        for t in topics:
          temp_df = self.df[self.df.topic == t].sort_values(by = 'importance', ascending=False).head(n).reset_index(drop=True)
          for i, row in temp_df.iterrows():
            print(f'{i}. '+self.short_descr(row.description))
      except:
        print("Не получается сократить новости.")
        topics = self.df.topic.unique()
        print(f"Cамые важные новости с {self.df.date.min()} по {self.df.date.max()}")
        for t in topics:
          print(f"По теме {t} произошло:")
          temp_df = self.df[self.df.topic == t].sort_values(by = 'importance', ascending=False).head(n).reset_index(drop=True)
          for i, row in temp_df.iterrows():
            print(f"{i+1}. {row.title}")

  def make_several_reports(self, test_dates_from, test_dates_to):
    for i in range(len(test_dates_to)):
      self.date_from = datetime.datetime.strptime(test_dates_from[i] + ' 00:00+0300', '%d-%m-%Y %H:%M%z')
      self.date_to = datetime.datetime.strptime(test_dates_to[i] + ' 00:00+0300', '%d-%m-%Y %H:%M%z')
      self.get_data()
      self.make_report()

In [ ]:
test_dates_from = ['03-05-2025', '07-09-2002', '09-07-2002', '09-07-2024']
test_dates_to = ['27-06-2025', '07-09-2003', '06-05-2025', '09-08-2024']
date_from = datetime.datetime.strptime(test_dates_from[3] + ' 00:00+0300', '%d-%m-%Y %H:%M%z')
date_to = datetime.datetime.strptime(test_dates_to[3] + ' 00:00+0300', '%d-%m-%Y %H:%M%z')

rep = Report(date_from, date_to, df)

In [ ]:
date_from = datetime.datetime.strptime('04-04-2024' + ' 00:00+0300', '%d-%m-%Y %H:%M%z')
date_to = datetime.datetime.strptime('04-12-2024' + ' 00:00+0300', '%d-%m-%Y %H:%M%z')

rep = Report(date_from, date_to, df)
rep.make_report()

Cамые важные новости с 2024-06-05 17:05:34+03:00 по 2024-12-03 19:39:17+03:00
По теме Медицина произошло:
1. С помощью томографии ученые впервые увидели, как псилоцибин перестраивает личность человека
2. Противовирусный препарат защитил от ВИЧ в 99% случаев
3. Внутривенные лекарства превратили в таблетки
4. Митохондрии вторглись в клетки головного мозга со своей ДНК
5. Первый в мире пациент получил вакцину от рака легких
По теме Психология произошло:
1. Бедность оказалась причиной одних заболеваний психики и следствием других
2. Психиатры выявили биологический маркер суицидальных мыслей
3. Просмотр развлекательных видео усилил скуку
4. Музыка Тейлор Свифт помогла фанатам справиться с расстройствами пищевого поведения
5. Чувствительные сотрудники оказались полезнее стрессоустойчивых
По теме Космонавтика произошло:
1. Школьники посадили секвойю, которая побывала в окололунном пространстве
2. Станция Lunar Gateway получается слишком тяжелой и сложной для посещения
3. Успешные испытания St

In [ ]:
date_from = input('Введите дату и время, начиная с которого нужно проанализировать новости. Формат mm-dd-yyyy hh:mm')
date_to = input('Введите дату и время, до которого нужно проанализировать новости. Формат mm-dd-yyyy hh:mm')
date_from = datetime.datetime.strptime(date_from + " 00:00+0300", "%m-%d-%Y %H:%M%z")
date_to = datetime.datetime.strptime(date_to + " 00:00+0300", "%m-%d-%Y %H:%M%z")
rep = Report(day2, day2, df, model=model)
rep.make_report()